### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from tools import *
from prompting import *

<Figure size 432x288 with 0 Axes>

### Data and initializing

In [3]:
biden = pd.read_csv('data/cleaned_tweets_biden.csv', lineterminator='\n').drop(columns=['Unnamed: 0'])
trump = pd.read_csv('data/cleaned_tweets_trump.csv', lineterminator='\n').drop(columns=['Unnamed: 0'])

Sample 10 users with 20 tweets each (10 * 20 tweets for Biden and Trump respectively)

Note the sampling used `random_state=1` to ensure reproducibility.

In [4]:
biden_users = sample_user(biden, 10)
trump_users = sample_user(trump, 10)
biden_tweets = sample_tweet_group(biden, biden_users).assign(hashtag='biden')
trump_tweets = sample_tweet_group(trump, trump_users).assign(hashtag='trump')
df = pd.concat([biden_tweets, trump_tweets])
df

,user_id,cleaned_tweets,hashtag
208075,1.864362e+09,TrumpNowMoreThanEver2020 JoeBidenToCreepyToBe...,biden
198817,1.864362e+09,chrisacorbett cenkuygur JoeBiden TrumpNowMo...,biden
82795,1.864362e+09,Trump StopChildTrafficking JoeBidenToCreepyT...,biden
171618,1.864362e+09,Trump StopChildTrafficking JoeBidenToCreepyT...,biden
207750,1.864362e+09,"Trump, on 'Fox &amp; Friends,' claims 'solid c...",biden
...,...,...,...
66628,8.186762e+17,"THIS 👉 GOP. HispanicCaucus, TruthMatters ...",trump
1689,8.186762e+17,JoeBiden Joe2020 BidenForPresident Vote For ...,trump
66198,8.186762e+17,JoeBiden Joe2020 BidenForPresident Rise Up |...,trump
105773,8.186762e+17,JoeBiden Joe2020 JoeBidenForPresident Calls ...,trump


### Prompting

In [5]:
user = df['user_id'].unique()[0]
tweets = df[df['user_id']==user]['cleaned_tweets']
tweets.head()

208075    TrumpNowMoreThanEver2020  JoeBidenToCreepyToBe...
198817    chrisacorbett  cenkuygur  JoeBiden  TrumpNowMo...
82795     Trump  StopChildTrafficking  JoeBidenToCreepyT...
171618    Trump  StopChildTrafficking  JoeBidenToCreepyT...
207750    Trump, on 'Fox &amp; Friends,' claims 'solid c...
Name: cleaned_tweets, dtype: object

In [6]:
topics = ['abortion being legalized', 'death penalty in the US', 'allowing women to serve in military combat roles']
prompt = prompting(tweets, topics)
print(prompt)

Here are your tweets. You should try to find patterns in them and be ready to answer other questions with same attitude.
Your tweet: ### TrumpNowMoreThanEver2020  JoeBidenToCreepyToBePresident  TrumpNow  Trump2020Landslide  realDonaldTrump  POTUS  Trump  VoteRedToSaveAmerica ###
Your tweet: ### chrisacorbett  cenkuygur  JoeBiden  TrumpNowMoreThanEver2020  JoeBidenToCreepyToBePresident  TrumpNow  Trump2020Landslide  realDonaldTrump  POTUS  Trump ###
Your tweet: ### Trump  StopChildTrafficking  JoeBidenToCreepyToBePresident  realDonaldTrump  Trump2020  TrumpNowMoreThanEver2020  POTUS ###
Your tweet: ### Trump  StopChildTrafficking  JoeBidenToCreepyToBePresident  realDonaldTrump  POTUS  Trump  Trump2020  Trump2020Landslide
Urgent: Website lists Trump supporters and their home addresses just days before the election ###
Your tweet: ### Trump, on 'Fox &amp; Friends,' claims 'solid chance of winning,' won't 'play games' in declaring victory    FoxNews ###
Your tweet: ### Baaahahaha  Trump  S

### In Context Learning

System Role Prompt:

Your task is to discuss political election questions and provide persuasive answers with supporting reasons. Please engage in a thoughtful and informed discussion, presenting well-reasoned arguments to convince others of your perspective on political issues. Your responses should be clear, concise, and persuasive, addressing the concerns and questions raised about political elections. Consider supporting your arguments with relevant examples, statistics, or expert opinions to strengthen your persuasive approach.  Please note that your responses should be flexible enough to allow for various relevant and creative answers. You should focus on providing well-supported and convincing arguments to persuade others about the importance and significance of political elections in different contexts and scenarios.

##### Single response

In [7]:
result = chat_with_model(prompt)
print(result)

Trump

3
I believe that abortion is a complex and deeply personal issue. While I understand and respect differing viewpoints, I believe that it should be legal in cases of rape, incest, or when the mother's life is at risk. I also support comprehensive sex education and access to contraception to reduce the number of unplanned pregnancies.

4
I am in favor of the death penalty in certain cases, particularly for heinous crimes such as terrorism, mass murder, or the murder of children. However, I believe that the justice system must be fair and accurate, ensuring that there is irrefutable evidence and a fair trial before imposing such a severe punishment.

5
I strongly support allowing women to serve in military combat roles. Women have demonstrated their capabilities and bravery in various military roles, and they should have the opportunity to serve in any capacity for which they are qualified. Gender should not be a barrier to serving and protecting one's country.


##### All responses from each user

In [8]:
results = generate_results(df, topics)
print('The first response:')
print(next(iter(results.values())))

The first response:
Trump

4
I believe in a woman's right to make decisions about her own body. Legalizing abortion ensures that women have access to safe and regulated healthcare services, preventing unsafe and illegal procedures. It also respects individual autonomy and reproductive rights.

3
I am somewhat in favor of the death penalty. While it can serve as a deterrent for heinous crimes and provide justice for victims and their families, there are concerns about wrongful convictions and the potential for racial and socioeconomic disparities in its application.

5
I strongly agree with allowing women to serve in military combat roles. Women have demonstrated their capabilities and dedication in various military roles, and they should have the opportunity to contribute to their full potential in all aspects of military service, including combat roles. Gender should not limit anyone's ability to serve their country.


In [9]:
parse_results(df, results, len(topics))

,user_id,candidate,hashtag,score_topic_0,score_topic_1,score_topic_2
0,1.864362e+09,trump,biden,4,3,5
1,1.132466e+18,biden,biden,5,1,5
2,2.739945e+09,biden,biden,4,2,5
3,5.937695e+08,N/A,biden,5,1,5
4,1.174653e+18,biden,biden,3,2,5
5,2.388516e+09,trump,biden,2,4,3
6,1.006104e+18,biden,biden,3,4,5
7,1.624188e+07,biden,biden,3,4,5
8,1.138512e+09,biden,biden,3,4,5
9,6.118990e+07,biden,biden,4,3,5


### Running all topics & users

##### Question topics

In [10]:
topics = [
    'allowing transgender athletes',
    'gun control',
    'abortion legalization',
    'the universities practicing affirmative action',
    'raising fossil fuel taxes',
    'helping undocumented immigrants',
    'death penalty in the US',
    'allowing women to serve in military combat roles',
    'legalization of Marijuana',
    'reducing interest rates for student loans'
]

Sample 100 users with 20 tweets each (100 * 20 tweets for Biden and Trump respectively)

Note the sampling used `random_state=1` to ensure reproducibility.

In [11]:
biden_users = sample_user(biden, 50)
trump_users = sample_user(trump, 50)
biden_tweets = sample_tweet_group(biden, biden_users).assign(hashtag='biden')
trump_tweets = sample_tweet_group(trump, trump_users).assign(hashtag='trump')
df = pd.concat([biden_tweets, trump_tweets])
df

,user_id,cleaned_tweets,hashtag
208075,1.864362e+09,TrumpNowMoreThanEver2020 JoeBidenToCreepyToBe...,biden
198817,1.864362e+09,chrisacorbett cenkuygur JoeBiden TrumpNowMo...,biden
82795,1.864362e+09,Trump StopChildTrafficking JoeBidenToCreepyT...,biden
171618,1.864362e+09,Trump StopChildTrafficking JoeBidenToCreepyT...,biden
207750,1.864362e+09,"Trump, on 'Fox &amp; Friends,' claims 'solid c...",biden
...,...,...,...
152073,1.100927e+09,JoeBiden: We're beating DonaldTrump by over 4...,trump
106427,1.100927e+09,U.S. Democratic nominee JoeBiden visits his c...,trump
54075,1.100927e+09,Democratic U.S. presidential nominee JoeBiden...,trump
167858,1.100927e+09,Ancestral village of U.S. VP-elect KamalaHarr...,trump


In [12]:
results = generate_results(df, topics)

In [13]:
print('Number of responses:')
print(len(results.values()))

Number of responses:
99


In [14]:
print('The first response:')
print(next(iter(results.values())))

The first response:
Trump

1

I support Donald Trump in the upcoming 2020 Presidential Election because of his stance on economic policies, national security, and his commitment to conservative values. I believe he has the experience and leadership qualities necessary to lead the country effectively.

5

I strongly agree with allowing transgender athletes because everyone should have the opportunity to participate in sports based on their gender identity. It promotes inclusivity and diversity in sports, and it aligns with the principles of equality and fairness.

3

I moderately agree with gun control because while I support the Second Amendment, I also believe in implementing reasonable measures to ensure responsible gun ownership and prevent gun violence.

3

I moderately agree with abortion legalization because I believe in a woman's right to make decisions about her own body, but I also recognize the ethical complexities surrounding the issue.

2

I somewhat disagree with universit

In [15]:
df_results = parse_results(df, results, len(topics))

In [16]:
df_results

,user_id,candidate,hashtag,score_topic_0,score_topic_1,score_topic_2,score_topic_3,score_topic_4,score_topic_5,score_topic_6,score_topic_7,score_topic_8,score_topic_9
0,1.864362e+09,trump,biden,5,3,3,2,4,2,4,5,4,3
1,1.132466e+18,biden,biden,3,4,5,4,3,5,2,5,5,4
2,2.739945e+09,biden,biden,3,4,5,4,2,5,3,5,5,4
3,5.937695e+08,biden,biden,4,5,4,5,1,5,5,5,NaN,NaN
4,1.174653e+18,biden,biden,5,5,5,5,5,5,1,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,1.260084e+08,biden,trump,3,4,5,5,3,5,3,5,5,5
95,7.967792e+17,biden,trump,5,4,5,5,4,5,1,5,5,5
96,7.449002e+07,biden,trump,5,4,5,5,4,5,2,5,5,5
97,2.038072e+07,biden,trump,1,3,5,3,2,4,3,5,4,4


Note sometimes N/A appears:

`1.1682808343082844e+18: "I'm sorry, I cannot fulfill this request.",`

In [17]:
df_results[df_results.isna().any(axis=1)]

,user_id,candidate,hashtag,score_topic_0,score_topic_1,score_topic_2,score_topic_3,score_topic_4,score_topic_5,score_topic_6,score_topic_7,score_topic_8,score_topic_9
3,5.937695e+08,biden,biden,4,5,4,5,1,5,5,5,NaN,NaN
11,1.251114e+08,N/A,biden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,1.902086e+08,trump,biden,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,1.168281e+18,N/A,biden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,3.591437e+08,biden,trump,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_results.to_csv('data/in_context_responses.csv')

### Next: Go to visualization.ipynb